In [ ]:
!pip install helper_utils

In [1]:
import chromadb # helper functions from utilities
from PyPDF2 import PdfReader # read the pdf files 
import os

In [2]:
reader = PdfReader("Report.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [5]:
split_text = text_splitter.split_text(' '.join(pdf_texts))


In [ ]:
split_text[3]

In [37]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=384)

tokens = []
for text in split_text:
    tokens += token_splitter.split_text(text)


In [38]:
tokens[3]

'infusing this technology across every layer of the tech stack. and, finally, we will continue to drive operating leverage, aligning our cost structur e with our revenue growth. as we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $ 211 billion in revenue and over $ 88 billion in operating income. a new era of ai there are two breakthroughs coming together to define this new era of ai. the first is the most universal interface : natural language. the long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human - computer interfaces — keyboards, mice, touch screens. we believe we have now arrived at the next big step forward — natural language — and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around us. the second is the emergence of a powerful new reasoning engine. for y ears, we ’ ve digitized daily life, places, and things'

In [7]:

from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

In [ ]:
embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([tokens[10]]))

In [9]:
client = chromadb.PersistentClient(path="db")

In [11]:
# Using Sentence Transformers Embeddings
collection = client.get_or_create_collection("report-2023", embedding_function=embedding_function)

In [20]:
# Using Google GenerativeAI Embeddings
from chromadb.utils.embedding_functions import GoogleGenerativeAiEmbeddingFunction
google_embed = GoogleGenerativeAiEmbeddingFunction(api_key=api_key)

collection2 = client.get_or_create_collection("report-2", embedding_function=google_embed)

In [21]:
ids = [str(i) for i in range(len(tokens))]

collection2.add(ids=ids, documents=tokens)
collection2.count()

318

In [33]:
result = collection.query(
    query_texts=["What is the total revenue"],
    n_results=2
)

print(result['documents'][0])

['each of these audits is not expected to be material to our consolidated financial statements. 68 note 13 — unearned revenue unearned revenue by segment was as follows : ( in millions ) june 30, 2023 2022 productivity and business processes $ 27, 572 $ 24, 558 intelligent cloud 21, 563 19, 371 more personal computing 4, 678 4, 479 total $ 53, 813 $ 48, 408 changes in unearned revenue were as follows : ( in millions ) year ended june 30, 2023 balance, beginning of period $ 48, 408 deferral of revenue 123, 935 recognition of unearned revenue ( 118, 530 ) balance, end of period $ 53, 813 revenue allocated to remaining performance obligations, which includes unearned revenue and amounts that will be invoiced and recognized as revenue in future periods, was $ 229 billion as of june 30, 2023, of which $ 224 billion is related', 'and are generally allocated based on relative gross margin. in addition, certain costs are incurred at a corporate level and allocated to our segments. these alloca

In [27]:
result = collection2.query(
    query_texts=["What is the total revenue at the end of 2023"],
    n_results=4
)

print("\n".join(result['documents'][0]))

income statements ( in millions, except per share amounts ) year ended june 30, 2023 2022 2021 revenue : product $ 64, 699 $ 72, 732 $ 71, 074 service and other 147, 216 125, 538 97, 014 total revenue 211, 915 198, 270 168, 088 cost of revenue : product 17, 804 19, 064 18, 219 service and other 48, 059 43, 586 34, 013 total cost of revenue 65, 863 62, 650 52, 232 gross margin 146, 052 135, 620 115, 856 research and development 27, 195 24, 512 20, 716 sales and marketing 22, 759 21, 825 20, 117 general and administrative 7, 575 5, 900 5, 107 operating income 88, 523 83, 383 69, 916 other income, net 788 333 1, 186 income before income taxes 89, 311 83, 716 71, 102 provision for income taxes 16, 950 10, 978 9, 831 net income $ 72, 361 $ 72, 738 $ 61, 271
2023 : ( in millions ) 2024 thereafter total long - term debt : ( a ) principal payments $ 5, 250 $ 47, 616 $ 52, 866 interest payments 1, 379 19, 746 21, 125 construction commitments ( b ) 12, 237 1, 218 13, 455 operating and finance le

In [ ]:
messages = [
    {"role": "System", }
]

In [12]:
import google.generativeai as genai
api_key = ""
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-pro')

In [39]:

def rag(query: str):

    retrieved_documents = collection2.query(
        query_texts=query,
        n_results=3
    )
    
    information = "\n\n".join(retrieved_documents['documents'][0])


    system_prompt = '''

    You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report.
    You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information.
    Otherwise say 'I do not know'.
    
    '''

    user_prompt = f"Question: {query}. \n Information: {information}. \n Answer:"
    
    prompt = f'{system_prompt} \n {user_prompt}'
    
    response = model.generate_content(f'{system_prompt} \n {user_prompt}')
    return response.text

    # return prompt

print(rag("What is the total revenue"))

211,915
